In [14]:
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent))
# print(str(Path().resolve().parent))
print(sys.path)
import os
os.environ['OMP_NUM_THREADS'] = "1"

['C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\kazum\\AppData\\Local\\Programs\\Python\\Python311', 'd:\\github\\scikit-quri\\pyenv', '', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages\\win32', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages\\win32\\lib', 'd:\\github\\scikit-quri\\pyenv\\Lib\\site-packages\\Pythonwin', 'D:\\github\\scikit-quri', 'D:\\github', 'D:\\github\\scikit-quri']


In [15]:
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent.parent))

# import better_exceptions
import numpy as np
from typing import Optional
from quri_parts.circuit.utils.circuit_drawer import draw_circuit
from quri_parts.circuit import UnboundParametricQuantumCircuit,LinearMappedUnboundParametricQuantumCircuit
from numpy.random import default_rng
from scikit_quri.circuit.circuit import LearningCircuit

from qulacs import Observable
from quri_parts.core.operator import Operator, pauli_label
from quri_parts.core.estimator.gradient import (
    create_parameter_shift_gradient_estimator,
    create_numerical_gradient_estimator,)
from quri_parts.qulacs.estimator import (
    create_qulacs_vector_concurrent_estimator,
    create_qulacs_vector_parametric_estimator,
    create_qulacs_vector_concurrent_parametric_estimator,
)
from quri_parts.algo.optimizer import Adam, LBFGS




In [16]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
x = df.loc[:, ["petal length (cm)", "petal width (cm)"]]
x_train,x_test,y_train,y_test = train_test_split(x,iris.target,test_size=0.25,random_state=0)

x_train = x_train.to_numpy()

from quri_parts.circuit.utils.circuit_drawer import draw_circuit
import matplotlib.pyplot as plt
n_qubits = 5
num_class = 3
print(x_train)

[[4.2 1.5]
 [4.  1.2]
 [5.5 2.1]
 [1.3 0.2]
 [5.1 2.3]
 [1.6 0.6]
 [1.5 0.2]
 [3.5 1. ]
 [5.5 1.8]
 [5.7 2.5]
 [5.  1.5]
 [5.8 1.8]
 [3.9 1.1]
 [6.1 2.3]
 [4.7 1.6]
 [3.8 1.1]
 [4.9 1.8]
 [5.1 1.5]
 [4.5 1.7]
 [5.  1.9]
 [4.7 1.4]
 [5.2 2. ]
 [4.5 1.6]
 [1.6 0.2]
 [5.1 1.9]
 [4.2 1.3]
 [3.6 1.3]
 [4.  1.3]
 [4.6 1.4]
 [6.  1.8]
 [1.5 0.2]
 [1.1 0.1]
 [5.3 1.9]
 [4.2 1.2]
 [1.7 0.2]
 [1.5 0.4]
 [4.9 1.5]
 [1.5 0.2]
 [5.1 1.8]
 [3.  1.1]
 [1.4 0.3]
 [4.5 1.5]
 [6.1 2.5]
 [4.2 1.3]
 [1.4 0.1]
 [5.9 2.1]
 [5.7 2.3]
 [5.8 2.2]
 [5.6 2.1]
 [1.6 0.2]
 [1.6 0.2]
 [5.1 2. ]
 [5.7 2.1]
 [1.3 0.3]
 [5.4 2.3]
 [1.4 0.2]
 [5.  2. ]
 [5.4 2.1]
 [1.3 0.2]
 [1.4 0.2]
 [5.8 1.6]
 [1.4 0.3]
 [1.3 0.2]
 [1.7 0.4]
 [4.  1.3]
 [5.9 2.3]
 [6.6 2.1]
 [1.4 0.2]
 [1.5 0.2]
 [1.4 0.2]
 [4.5 1.3]
 [4.4 1.4]
 [1.2 0.2]
 [1.7 0.5]
 [4.3 1.3]
 [1.5 0.4]
 [6.9 2.3]
 [3.3 1. ]
 [6.4 2. ]
 [4.4 1.4]
 [1.5 0.1]
 [4.8 1.8]
 [1.2 0.2]
 [6.7 2. ]
 [1.5 0.3]
 [1.6 0.2]
 [6.1 1.9]
 [1.4 0.2]
 [5.6 2.4]
 [4.1 1.3]
 [3.9 1.2]

In [17]:
from scikit_quri.circuit.pre_defined import create_qcl_ansatz,create_farhi_neven_ansatz
# circuit = create_farhi_neven_ansatz(n_qubits, 2,0)
circuit = create_qcl_ansatz(n_qubits,3,1.)
draw_circuit(circuit.circuit)
print(circuit.circuit.parameter_count)

   ___     ___     ___     ___     ___     ___     ___     ___     ___     ___  
  |PRY|   |PRZ|   |Mat|   |PRX|   |PRZ|   |PRX|   |Mat|   |PRX|   |PRZ|   |PRX| 
--|0  |---|1  |---|10 |---|11 |---|12 |---|13 |---|26 |---|27 |---|28 |---|29 |-
  |___|   |___|   |   |   |___|   |___|   |___|   |   |   |___|   |___|   |___| 
   ___     ___    |   |    ___     ___     ___    |   |    ___     ___     ___  
  |PRY|   |PRZ|   |   |   |PRX|   |PRZ|   |PRX|   |   |   |PRX|   |PRZ|   |PRX| 
--|2  |---|3  |---|   |---|14 |---|15 |---|16 |---|   |---|30 |---|31 |---|32 |-
  |___|   |___|   |   |   |___|   |___|   |___|   |   |   |___|   |___|   |___| 
   ___     ___    |   |    ___     ___     ___    |   |    ___     ___     ___  
  |PRY|   |PRZ|   |   |   |PRX|   |PRZ|   |PRX|   |   |   |PRX|   |PRZ|   |PRX| 
--|4  |---|5  |---|   |---|17 |---|18 |---|19 |---|   |---|33 |---|34 |---|35 |-
  |___|   |___|   |   |   |___|   |___|   |___|   |   |   |___|   |___|   |___| 
   ___     ___    |   |    _

In [18]:
print(f"{x_train[3]=}")
# init_params = [-0.6459906318187236, 3.5726052125794032, -3.7261666316598814, -6.381331778040319, 4.448677843102995, 3.4708870076700777, -5.070968615387754, -6.328852747092929]
# init_params =[4.002148315014479, 1.6951199159934145, 0.25744424357926954, 0.10384619671527331, 5.109927617709579, 5.735012432197602, 3.811604993109835, 4.583562073612696, 3.415696558991173, 5.8752333142921085, 5.126159064066656, 0.017206504032783308, 5.3872299529679575, 0.21102439329246717, 4.584560380312186]
init_params = [np.random.rand() for _ in range(circuit.n_learning_params)]
unbound_circuit = circuit.bind_input_and_parameters(x_train[0],init_params)
for gate in unbound_circuit.gates:
    print(gate.name,gate.params)
    if gate.name == "UnitaryMatrix":
        # print(gate.unitary_matrix)
        pass

x_train[3]=array([1.3, 0.2])
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
RY (1.5707963267948966,)
RZ (0.0,)
UnitaryMatrix ()
RX (0.998829664375149,)
RZ (0.74218214575179,)
RX (0.942415836739186,)
RX (0.46975794471997845,)
RZ (0.80049837823482,)
RX (0.6565288124714608,)
RX (0.39654688875100497,)
RZ (0.1123536399378482,)
RX (0.8006586558732287,)
RX (0.7555350433065343,)
RZ (0.32015484922750637,)
RX (0.7711988553032952,)
RX (0.8467495135234959,)
RZ (0.36157242746294294,)
RX (0.266976131200916,)
UnitaryMatrix ()
RX (0.9526317825074349,)
RZ (0.8033664163858633,)
RX (0.036258557028626015,)
RX (0.6153052233208129,)
RZ (0.7366447123706484,)
RX (0.6337327764927418,)
RX (0.3638746307040641,)
RZ (0.2751206434533161,)
RX (0.6298532261915077,)
RX (0.8759640975101645,)
RZ (0.8279861725874151,)
RX (0.8152888743456241,)
RX (0.15253187084797548,)
RZ (0.17411650385190647,)
RX (0.35436169919881755,)
UnitaryMat

In [19]:
from quri_parts.core.state import quantum_state
param_circuit_state = quantum_state(n_qubits=5,circuit=circuit.circuit)
params = circuit.generate_bound_params(x_train[0],init_params)
circuit_state = param_circuit_state.bind_parameters(params)

from quri_parts.qulacs.estimator import create_qulacs_vector_estimator
from quri_parts.core.operator import Operator, pauli_label

op = Operator({pauli_label("Z0"): 1.0})
# estimator = create_qulacs_vector_estimator()
estimator = create_qulacs_vector_concurrent_estimator()
estimate = create_qulacs_vector_estimator()(op, circuit_state)
print(f"{estimate.value.real*2.2=}")

estimate.value.real*2.2=-0.34793999096071737


In [20]:
from scikit_quri.qnn.classifier import QNNClassifier

ops = []
for i in range(num_class):
    op = Operator({pauli_label(f"Z {i}"):1.0})
    ops.append(op)

adam = Adam()
estimator = create_qulacs_vector_concurrent_estimator()
gradient_estimator = create_parameter_shift_gradient_estimator(create_qulacs_vector_concurrent_parametric_estimator())
qnn = QNNClassifier(circuit,num_class,estimator,gradient_estimator,adam,ops,)

In [21]:
res = qnn._predict_inner(x_train,params=init_params)
print(res)

[[-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.56419279  0.00538583  0.73099975]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.63494577 -0.00412106  0.49008006]
 [-0.56419279  0.00538583  0.73099975]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.56419279  0.00538583  0.73099975]
 [-0.34793999 -0.33352899  0.21768513]
 [-0.34793999 -0.33352899

In [22]:
print(circuit.get_learning_param_indexes())
res = qnn.cost_func_grad(x_train,y_train,init_params)


[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
         10952578 function calls in 4.143 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.013    0.013    4.158    4.158 classifier.py:187(_estimate_grad)
      336    0.014    0.000    4.093    0.012 gradient.py:190(estimator)
      336    0.212    0.001    4.079    0.012 gradient.py:117(parameter_shift_gradient_estimates)
      336    0.001    0.000    2.390    0.007 estimator.py:214(estimator)
      336    0.003    0.000    2.389    0.007 concurrent.py:23(execute_concurrently)
      336    0.456    0.001    2.386    0.007 estimator.py:168(_sequential_parametric_estimate)
  2032800    0.841    0.000    0.841    0.000 {built-in method qulacs_core.set_parameter}
      336    0.010    0.000    0.755    0.002 gradient.py:144(<listc

In [23]:
qnn.fit(x_train,y_train,maxiter=500)

init_params=array([0.64446437, 4.14524559, 2.86021462, 5.56089524, 5.59136271,
       2.45585758, 2.51188315, 5.92857801, 1.22345984, 4.29921178,
       2.1395672 , 2.26059828, 3.20557646, 5.94602819, 2.0341128 ,
       5.38031042, 2.60374511, 3.44387714, 4.68271527, 1.33633554,
       3.3548388 , 3.35641937, 0.96993052, 4.94183119, 4.83793865,
       2.21455815, 0.72781661, 3.06313189, 3.4770224 , 3.51612158,
       6.24063638, 2.8419634 , 3.914744  , 5.84551234, 4.54756318,
       5.3604713 , 4.45795126, 5.29776882, 5.03728253, 5.1520191 ,
       2.86543982, 2.73940571, 2.42402399, 2.3898319 , 4.10491057])
params[:4]=array([0.64446437, 4.14524559, 2.86021462, 5.56089524])
cost_func(init_params)=1.0933191937556452
self._estimate_grad(x_scaled,init_params)=array([[[ 5.17539989e-01,  4.48201431e-02, -3.46092323e-01, ...,
          7.97972799e-17, -1.04083409e-17, -1.04083409e-17],
        [-1.36875717e-01,  6.45034443e-02, -1.52165430e-02, ...,
         -1.11022302e-16, -2.77555756e-17,

KeyboardInterrupt: 

In [ ]:
y_pred = qnn.predict(x_test)
print(y_pred)
print(y_test)
from scipy.special import softmax
from sklearn.metrics import log_loss

y_pred_sm = softmax(y_pred,axis=1)
print(y_pred_sm)
log = log_loss(y_test,y_pred_sm)
print(log)

[[-1.00032576 -1.34144928 -0.2319797 ]
 [-0.90820358  0.53196672 -0.52652201]
 [ 0.27904822  0.25731059 -0.51303696]
 [-0.28550018  0.17303456  1.15248768]
 [ 0.21774731  0.31618236 -0.45274049]
 [-1.1503741  -0.6965795  -0.70923896]
 [ 0.35662233  0.14530502 -0.69633455]
 [-0.97280635  0.11705081 -0.08499152]
 [-0.97257435  0.18989272 -0.05976834]
 [-0.91594141  0.27755931 -0.46218168]
 [-0.76841235  0.21573364  0.54574387]
 [-0.96984339  0.12316847 -0.19858934]
 [-0.98208034  0.34730218 -0.19782887]
 [-0.97298319  0.12001232 -0.14391651]
 [-0.97571877  0.19229539 -0.12124181]
 [ 0.12842814  0.28161275 -0.09496872]
 [-0.96984339  0.12316847 -0.19858934]
 [-0.96641272  0.35311581 -0.34468643]
 [ 0.30289591  0.21256048 -0.62071676]
 [ 0.34899118  0.09664571 -0.74116183]
 [-0.95431051 -0.48444131  0.11900722]
 [-0.96984339  0.12316847 -0.19858934]
 [-0.04315306  0.50813761 -0.32938078]
 [ 0.27904822  0.25731059 -0.51303696]
 [-0.96197017 -0.1755576   0.05938136]
 [ 0.2952473  -0.07117232

d:\github\scikit-quri\pyenv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
